<a href="https://colab.research.google.com/github/Jungseunggi/Section4_week2/blob/main/Section4_week2_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4. 트랜스포머 (Transformer)**

**트랜스포머란??**

* attention mechanism은 seq2seq의 입력 시퀀스 정보 손실을 보정해주기 위해 사용됨

* attention mechanism을 보정 목적이 아닌, 인코더와 디코더로 구성한 모델이 바로 트랜스포머

* 트랜스포머는 RNN을 사용하지 않고 인코더와 디코더를 설계하였으며, 성능도 RNN보다 우수함

* RNN을 사용하지 않는 이유는 구조적으로 단어가 순차적 입력을 받기 때문에 시퀀스가 길수록 연산 시간이 길어짐

* 논문 제목이 'Attention Is All You Need' 것을 보면 Attention 메카니즘을 활용

## **4.1 기존 Seq2Seq모델의 문제점**

<img src='https://velog.velcdn.com/images/ny_/post/b5142c0b-597f-430f-a3c4-b1bc053018d4/image.png' width=800>

<sub> 이미지 출처 : https://velog.io/@ny_/Transformer-Attention-is-all-you-need</sub>

* context vector $v$에 소스 문장의 정보를 압축
* 따라서 병목(bottleneck)이 발생 -> 성능 하락
* RNN과 비교해 LSTM에서 장기 의존성(long-term dependency)을 개선했지만 여러 layer를 거치면 여전히 앞쪽에 등장한 정보는 손실될 수 있으며, 긴 문장 예측이 어려운 단점


<img src='https://velog.velcdn.com/images/ny_/post/0ee1360d-ec58-4b06-877e-7bc646556914/image.png' width=800>

<sub> 이미지 출처 : https://velog.io/@ny_/Transformer-Attention-is-all-you-need</sub>

* 따라서 문장이 길어질수록 context vector가 손실되지 않도록 매번 RNN셀에 context vector를 참고하는 방법 제시

* 하지만 여전히 하나의 input sequence를 고정된 크기의 context vector로 압축시켜야 하기 때문에, 성능이 저하


## **4.2 Seq2Seq Model with Attention(어텐션 메커니즘 적용)**

<img src='https://velog.velcdn.com/images/ny_/post/f519c104-4c86-4c61-b3a4-e1c0df17d2c9/image.png' width=800>

<sub> 이미지 출처 : https://velog.io/@ny_/Transformer-Attention-is-all-you-need</sub>

* 그렇다면 매번 소스 문장에서의 출력 전부를 입력으로 받으면 어떨까?(GPU 성능 상승)

* 디코더는 context vector 뿐만 아니라 인코더의 모든 $h$(hidden states)를 전부 반영한 weighted sum vector w를 사용해 출력을 생성(인코더의 모든 출력을 참고)



## **4.3 Self-attention**

* 단어 간의 관계성 연산 결과를 활용하여 연관성이 높은 단어끼리 연결해주기 위해 활용하는 것

* 단어에 맥락을 불어 넣어주는 Method

* 쿼리(Q)는 분석하고자 하는 단어에 대한 가중치 벡터
* 키(K)는 각 단어가 쿼리에 해당하는 단어와 얼마나 연관있는 지를 비교하기 위한 가중치 벡터
* 밸류(V)는 각 단어의 의미를 살려주기 위한 가중치 벡터

* 순서
 
 * 단어 행렬과 준비된 가중치 행렬을 내적하여 Q,K,V 행렬을 만듬
 
 * Q 행렬과 K 행렬을 내적, 내적을 통해 나오는 값이 Attention 스코어(Score)
 
 * 트랜스포머에서는 이 가중치를 q,k,v 벡터 차원  $d_k$ 의 제곱근인 $\sqrt{d_k}$ 
계산값을 안정적으로 만들어주기 위한 계산 보정

 * 계산하여 나온 값에 softmax 함수를 취함, 해당하는 단어와 문장 내 다른 단어가 가지는 관계의 비율을 구할 수 있음(유사성 확인)

 * 마지막으로 밸류(V) 각 단어의 벡터를 곱해준 후 모두 더하면 Self-Attention 과정이 마무리

<img src="https://user-images.githubusercontent.com/45377884/112799904-ecb3a100-90a9-11eb-9072-87a965e81a77.png" width=300>

<sub>이미지 출처 : https://proceedings.neurips.cc/paper/2017/hash/3f5ee243547dee91fbd053c1c4a845aa-Abstract.html</sub>


Attention과 비슷하지만, Transformer의 Self-Attention과 Attention 사이에는 몇가지 차이가 존재

**Attention VS Self-Attention** 

|| Attention | Self-Attention |
|-|-|-|
|**Q,K,V**| Q는 Decoder Cell에서 도출되고, K와 V는 Encoder Cell에서 도출| Q, K, V는 모두 동일한 Vector(Embedding Vector)에서 도출|
|**Time Step**| 활용 | 활용하지 않음|
|**Long-term Dependency**| Hidden Layer 연산 결과(Hidden State Vector)를 다음 Model에 적용시키는 방식으로 이전 단어 정보를 전달| Encoder는 모든 단어에 대하여, Decoder는 이전에 예측했던 모든 단어에 대하여 Q(Hidden Vector)를 구하여, 이 값을 온전히 활용|
||시간이 지남에 따라 어쩔 수 없이 초기 해석 정보가 삭제|Decoder측의 Length가 길어지더라도 초기 Decoder의 해석 정보가 지워지지 않고 온전히 반영될 수 있다는 장점|
|**방향성**|Unidirectional(단방향)|Bidirectional(양방향)|
||전 단어를 예측하면서 나온 Hidden State Vector를 활용하여 현재 단어를 예측|모든 위치에 존재하는 단어와 연산이 진행|


**Vector Self_Attention 과정**

1. **Input 단어들을 활용하여 Embdding Vector 도출**
 * 단어 1개당 Embedding Vector 1개씩 도출
2. **Embedding Vector를 활용하여 Queries, Keys, Values Vector를 생성**
 * Embedding Vector 1개당 (Q, K, V) 쌍 1개씩 도출됨


<img src="https://velog.velcdn.com/images%2Fidj7183%2Fpost%2F64e9ee88-753b-4890-86bf-186584fd8bf0%2Fimage.png"  width="500" />

<sub>이미지 출처 : https://velog.io/@idj7183/Attention-TransformerSelf-Attention</sub>

3. **Q, K, V를 활용하여 점수 계산**
 * 특정 단어의 Query Vector를 활용하여 모든 단어의 Key Vector와 내적을 통해 Score를 구함
 * Score : 해당 단어에 대한 Encode를 수행할 때, 다른 단어들에 얼마나 집중해야 할지를 결정해줌


<img src="https://velog.velcdn.com/images%2Fidj7183%2Fpost%2Fbba0a682-946e-4e7a-b911-4559395e5f90%2Fimage.png"  width="500" />

<sub>이미지 출처 : https://velog.io/@idj7183/Attention-TransformerSelf-Attention</sub>


4. **Score를  $\sqrt{d_k}$로 나눠줌**

<img src="https://velog.velcdn.com/images%2Fidj7183%2Fpost%2F13be5d99-9046-44f3-ab2a-0677e9c39823%2Fimage.png"  width="500" />

<sub>이미지 출처 : https://velog.io/@idj7183/Attention-TransformerSelf-Attention</sub>


5. **나눠준 모든 값을 Softmax 함수에 통과시켜줌**

6. **Softmax 연산 결과값들을 Value 벡터에 곱해줌**

7. **Value 벡터를 곱해준 모든 Vector들을 더해줘 나오는 Vector가 해당 단어가 Self-Attention을 거친 이후 반환할 Self-Attention Layer의 출력값이 됨**
<img src="https://velog.velcdn.com/images%2Fidj7183%2Fpost%2F976402cd-eff9-4e4b-8428-75fd91e088b3%2Fimage.png" width="500" />

<sub>이미지 출처 : https://velog.io/@idj7183/Attention-TransformerSelf-Attention</sub>


###**4.3.1 Positional Encoding (위치 인코딩)**

* 단어의 **상대적인 위치 정보를 담은 벡터**를 만드는 과정
* 기존의 RNN은 단어의 위치를 따라 순차적으로 입력받아 단어의 위치정보를 활용할 수 있었음
* 트랜스포머의 경우, RNN을 활용하지 않았기 때문에 단어의 위치정보를 다른 방식으로 줄 필요가 있음
* 이를 위해 각 단어의 임베딩 벡터에 위치 정보들을 더하게 되는데 이를 포지셔널 인코딩이라 함
* 보통 포지셔널 인코딩은 sin, cos을 이용하여 계산(따라서 -1 ~ 1 사이값)

$$
\begin{aligned}
\text{PE}_{\text{pos},2i} &= \sin \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg) \\
\text{PE}_{\text{pos},2i+1} &= \cos \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg)
\end{aligned}
$$

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def positional_encoding(dim, sentence_length):  # dim 임베딩벡터의 차원수
  encoded_vec = np.array([pos / np.power(10000, 2*i/dim) for pos in range(sentence_length) for i in range(dim)]) # i:index, pos:position
  encoded_vec[::2] = np.sin(encoded_vec[::2])  #짝수 [A:B:C]의 의미는, index A 부터 index B 까지 C의 간격
  encoded_vec[1::2] = np.cos(encoded_vec[1::2]) #홀수
  return tf.constant(encoded_vec.reshape([sentence_length, dim]), dtype = tf.float32) # tf.constant 상수선언?? 

### **4.3.2 Multi Head Attetion(encoder)**


<img width="300" alt="self-Attn" src="https://user-images.githubusercontent.com/45377884/112809266-ca735080-90b4-11eb-9a25-7f34f37880c7.png">

* 여러 개의 Attention 메커니즘을 동시에 병렬적으로 실행

* 여러명 대려와서 번역시키는거

* 문장 내에 단어간의 유사도를 확인

<img src="http://jalammar.github.io/images/t/transformer_attention_heads_z.png" width="500"/>

###**4.3.3 Layer Normalization & Skip Connection**

<img width="300" alt="lnorm_resicon" src="https://user-images.githubusercontent.com/45377884/113169444-9056aa00-9280-11eb-8ba0-17c9211ad412.png">

* 트랜스포머의 모든 sub-layer에서 출력된 벡터는 Layer normalization과 Skip connection을 거치게 됨
* 우선은 계산빠르게 해주는 과정이라고 알고 넘어가자

In [ ]:
def layer_norm(inputs, eps=1e-6):
  feature_shape = inputs.get_shape()[-1:]
  mean = tf.keras.backed.mean(inputs,[-1], keepdims = True)
  std = tf.keras.backed.std(inputs,[-1], keepdims = True)
  beta = tf.Variable(tf.zeros(feature_shape), trainable = False)
  gamma = tf.Variable(tf.ones(feature_shape), trainable = False)
  return gamma*(inputs-mean)/(std+eps) + beta  # eps를 더해주는 이유는 분모가 0이되선 안되므로 아주작은 값을 더해줌


In [ ]:
# tf.constant, get_shape 예시

scalarTensor     = tf.constant(100)
vectorTensor     = tf.constant([1, 2, 3, 4, 5])
matrixTensor     = tf.constant([[1, 2, 3], [4, 5, 6]])
cubeMatrixTensor = tf.constant([[[1], [2], [3]], [[4], [5], [6]], [[7], [8], [9]]])

scalarTensorShape     = scalarTensor.get_shape()
vectorTensorShape     = vectorTensor.get_shape()
matrixTensorShape     = matrixTensor.get_shape()
cubeMatrixTensorShape = cubeMatrixTensor.get_shape()

print(scalarTensorShape    )
print(vectorTensorShape    )
print(matrixTensorShape    )
print(cubeMatrixTensorShape)

### **4.3.4 Feed Forward Neural Network**

<img width="300" alt="스크린샷 2021-03-29 오후 5 27 32" src="https://user-images.githubusercontent.com/45377884/112808809-58027080-90b4-11eb-8ca7-ffa38e577d3d.png">



* 은닉층의 차원이 늘어났다가 다시 원래 차원으로 줄어드는 단순한 2층 신경망

* 활성화 함수(Activation function)으로 ReLU를 사용

###**4.3.5 Masked Self-Attention**

<img width="300" alt="Masked_Self-Attention_in_structure" src="https://user-images.githubusercontent.com/45377884/112808936-78322f80-90b4-11eb-9315-22cd9caad41d.png">

* 해당 위치 타깃 단어 뒤에 위치한 단어는 Self-Attention에 영향을 주지 않도록 마스킹(Masking)을 해줌

* 뒤에 단어를 보고 번역하면 컨닝(따라서 뒤에 단어는 지워줌)

* 컨닝방지

<img src="http://jalammar.github.io/images/gpt2/self-attention-and-masked-self-attention.png" width="500">

<img width="600"  src="http://jalammar.github.io/images/gpt2/transformer-attention-mask.png">

<img width="600"  src="http://jalammar.github.io/images/gpt2/transformer-attention-masked-scores-softmax.png">

###**4.3.6 Multi Head Attention(decoder)**

* 이 층에서는 디코더 블록의 Masked Self-Attention으로부터 출력된 벡터를 쿼리(Q) 벡터로 사용

* 키(K)와 밸류(V) 벡터는 최상위(=6번째) 인코더 블록에서 사용했던 값을 그대로 가져와서 사용

* Encoder-Decoder Attention 가 진행되는 순서를 나타낸 이미지

<img width="700" alt="Encoder-Decoder_Attention_gif" src="http://jalammar.github.io/images/t/transformer_decoding_1.gif">

##**4.3.7 Linear & Softmax Layer**

<img width="300" alt="Linear_Softmax" src="https://user-images.githubusercontent.com/45377884/112815762-994a4e80-90bb-11eb-9a57-a8be65c1a30b.png">

디코더의 최상층을 통과한 벡터들은 Linear 층을 지난 후 Softmax를 통해 예측할 단어의 확률을 구함